# Librerias

In [1]:
%pip install bayesian-optimization

  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from scipy.stats import skew, pearsonr, spearmanr
import plotly.graph_objects as go
import plotly.figure_factory as ff
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

# Cargar los Datos

In [3]:
file_path = 'data_treino_dv_df_2000_2010_new.csv'
df_wind = pd.read_csv(file_path)
df_wind.head()

,Hora_UTC,Direccion_Viento_Grados,Velocidad_Viento_m_s,Humedad_Max_Hora_Anterior,Humedad_Min_Hora_Anterior,Temperatura_Max_Hora_Anterior_C,Temperatura_Min_Hora_Anterior_C,Humedad_Relativa_Horaria,Presion_Atmosferica_Estacion_mB,Precipitacion_Total_Horario_mm,Viento_Rajada_Maxima_m_s,Presion_Atmosferica_Max_Hora_Anterior_mB,Presion_Atmosferica_Min_Hora_Anterior_mB
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5


In [76]:
def calculate_vif(df):
    """
    Calcula el VIF para cada variable en el DataFrame proporcionado.
    """
    VIF = pd.DataFrame()
    df = add_constant(df)  # Añade una constante para el cálculo del VIF
    VIF["Variable"] = df.columns
    VIF["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    VIF = VIF[VIF["Variable"] != "const"]
    return VIF

df_numeric = df_wind.select_dtypes(include=['float64', 'int64']).dropna()

# Eliminar la variable de respuesta 'VelVientoH' si está presente
if 'Velocidad_Viento_m_s' in df_numeric.columns:
    df_numeric = df_numeric.drop('Velocidad_Viento_m_s', axis=1)

# Calcular el VIF inicial
vif_data = calculate_vif(df_numeric)

# Para registrar las variables eliminadas y sus VIFs
eliminados = []

# Reducir la dimensionalidad eliminando columnas con VIF ≥ 5
while vif_data['VIF'].max() >= 10:
    max_vif_var = vif_data.sort_values('VIF', ascending=False).iloc[0]
    print(f"Eliminando {max_vif_var['Variable']} con VIF: {max_vif_var['VIF']}")
    eliminados.append(max_vif_var)

    df_numeric = df_numeric.drop(max_vif_var['Variable'], axis=1)

    vif_data = calculate_vif(df_numeric)

# Visualizar el VIF final de las variables restantes
# fig = go.Figure([go.Bar(x=vif_data['Variable'], y=vif_data['VIF'], text=vif_data['VIF'], textposition='auto')])
# fig.update_layout(title_text='VIF de las Variables Restantes', xaxis_title='Variable', yaxis_title='VIF')
# fig.show()
vif_data


Eliminando Humedad_Min_Hora_Anterior con VIF: 80.56035297652929
Eliminando Humedad_Relativa_Horaria con VIF: 46.37960517199195
Eliminando Temperatura_Max_Hora_Anterior_C con VIF: 22.196504568127665
Eliminando Presion_Atmosferica_Min_Hora_Anterior_mB con VIF: 16.065510383402767


,Variable,VIF
1,Direccion_Viento_Grados,1.286314
2,Humedad_Max_Hora_Anterior,2.130640
3,Temperatura_Min_Hora_Anterior_C,1.978843
4,Presion_Atmosferica_Estacion_mB,4.547480
5,Precipitacion_Total_Horario_mm,1.064873
6,Viento_Rajada_Maxima_m_s,1.250473
7,Presion_Atmosferica_Max_Hora_Anterior_mB,4.719906


In [ ]:
# df_wind_model.drop(['Hora_UTC'], axis = 1, inplace = True)

In [7]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt

# Definición de la función de rolling window kfolds
def rolling_window_kfolds(lista, window, jump):
    folds = len(lista) - window
    results = []
    for fold in range(folds):
        start_idx = fold
        end_idx = start_idx + window
        data_test_idx = end_idx
        data = lista[start_idx:end_idx]
        data_test = lista[data_test_idx:data_test_idx + 1]  
        results.append((data, data_test))
    return results

# Datos de ejemplo - asumiendo que df_wind ya está definido
new_vars = ['Humedad_Min_Hora_Anterior', 'Humedad_Relativa_Horaria',
            'Viento_Rajada_Maxima_m_s', 'Direccion_Viento_Grados',
            'Velocidad_Viento_m_s']
df_wind_model = df_wind.loc[:, new_vars]

# Preparación de datos
X = df_wind_model.drop(['Velocidad_Viento_m_s'], axis=1)
y = df_wind_model['Velocidad_Viento_m_s']
indices = list(range(len(df_wind_model)))
rolling_indices = rolling_window_kfolds(indices, window=7, jump=1)

# Modelos
models = {
    'Decision Tree': DecisionTreeRegressor(max_depth=10),
    # 'XGBoost': XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=10, reg_alpha=0.01, reg_lambda=1),
    'Bayesian Regression': BayesianRidge(alpha = 0.1),
    'Ridge': Ridge(alpha=0.1)
}


model_results = {}
for model_name, model in models.items():
    rmses = []
    print(f"Training {model_name} using Rolling Window K-Folds:")
    for train_idx, test_idx in tqdm(rolling_indices, desc=f"Progress for {model_name}"):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        if model_name == 'XGBoost':
            model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=False)
        else:
            model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        rmse = sqrt(mean_squared_error(y_test, y_pred))
        rmses.append(rmse)
    
    model_results[model_name] = {'Average RMSE': np.mean(rmses)}

# Imprimir los resultados
for model_name, results in model_results.items():
    print(f'{model_name} - Average RMSE: {results["Average RMSE"]}')

Training Decision Tree using Rolling Window K-Folds:


Progress for Decision Tree:   0%|          | 0/87686 [00:00<?, ?it/s]

Training Bayesian Regression using Rolling Window K-Folds:


Progress for Bayesian Regression:   0%|          | 0/87686 [00:00<?, ?it/s]

Training Ridge using Rolling Window K-Folds:


Progress for Ridge:   0%|          | 0/87686 [00:00<?, ?it/s]

Decision Tree - Average RMSE: 0.7148313299728577
Bayesian Regression - Average RMSE: 0.7493329882046217
Ridge - Average RMSE: 0.826293410134576


In [50]:
# Selecting features for the model
new_vars = ['Humedad_Min_Hora_Anterior', 'Humedad_Relativa_Horaria',
            'Viento_Rajada_Maxima_m_s', 'Direccion_Viento_Grados','Humedad_Max_Hora_Anterior',
            'Velocidad_Viento_m_s']

df_wind_model = df_wind.loc[:, new_vars]

X = df_wind_model.drop(['Velocidad_Viento_m_s'], axis=1)  
y = df_wind_model['Velocidad_Viento_m_s']

# Función para generar índices utilizando la técnica de ventana rodante
def rolling_window_kfolds(lista, window, jump):
    folds = len(lista) - window
    results = []

    for fold in range(folds):
        start_idx = fold
        end_idx = start_idx + window
        end_idx_test = start_idx + 1

        data = lista[start_idx:end_idx]
        target = lista[end_idx:end_idx + window]
        data_test = lista[start_idx:end_idx_test]
        target_test = lista[end_idx:end_idx + jump]

        results.append({'window': data, 'target': target, 'data_test': data_test, 'target_test': target_test})
        
        if fold == (len(lista)//window) - 1:
            break

    return results

lista = [i for i in range(len(df_wind_model))]  # Lista para obtener los indices
results = rolling_window_kfolds(lista, window =7, jump=1)

# Definición de modelos y hiperparámetros
models = {
    'XGBoost': Pipeline([('scaler', StandardScaler()), ('model', XGBRegressor(objective='reg:squarederror'))]),
    'Bayesian Regression': Pipeline([('scaler', StandardScaler()), ('model', BayesianRidge())]),
    'Ridge': Pipeline([('scaler', StandardScaler()), ('model', Ridge())]),
    'KNN': Pipeline([('scaler', StandardScaler()), ('model', KNeighborsRegressor())]),
    'Random Forest': Pipeline([('scaler', StandardScaler()), ('model', RandomForestRegressor())]),
    'Lasso': Pipeline([('scaler', StandardScaler()), ('model', Lasso())])
}

hyperparams = {
    'XGBoost': {'model__n_estimators': [50, 100], 'model__learning_rate': [0.01, 0.05], 'model__max_depth': [3, 5], 'model__reg_alpha': [0.01, 0.1], 'model__reg_lambda': [0.5, 1]},
    'Ridge': {'model__alpha': [0.1]},
    'Bayesian Regression': {'model__alpha_1': [1e-5], 'model__alpha_2': [1e-5]},
    'KNN': {'model__n_neighbors': [3, 5, 7]},
    'Random Forest': {'model__n_estimators': [100], 'model__max_depth': [3, 5, 10]},
    'Lasso': {'model__alpha': [0.01, 0.1]}
}

model_results = {}

# Bucle sobre los modelos y sus hiperparámetros
for model_name, params in hyperparams.items():
    for param_name, values in params.items():
        for value in tqdm(values, desc=f'Training {model_name} - {param_name}'):
            rmses = []

            for result in results:
                X_train, y_train = df_wind_model.iloc[result['window']], df_wind_model.iloc[result['target']]['Velocidad_Viento_m_s']
                X_test, y_test = df_wind_model.iloc[result['data_test']], df_wind_model.iloc[result['target_test']]['Velocidad_Viento_m_s']

                model = models[model_name]
                model.set_params(**{param_name: value})
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                rmse = sqrt(mean_squared_error(y_test, y_pred))
                rmses.append(rmse)

            mean_rmse = np.mean(rmses)
            if (model_name, param_name) not in model_results or mean_rmse < model_results[(model_name, param_name)][1]:
                model_results[(model_name, param_name)] = (value, mean_rmse)

# Imprimir los mejores parámetros y su RMSE para cada configuración de modelo
for (model_name, param_name), (best_value, min_rmse) in model_results.items():
    print(f'{model_name} [{param_name}={best_value}] - Minimum RMSE: {min_rmse}')

Training Lasso - model__alpha: 100%|██████████| 2/2 [01:42<00:00, 51.39s/it]

XGBoost [model__n_estimators=100] - Minimum RMSE: 0.0006141912277945157
XGBoost [model__learning_rate=0.05] - Minimum RMSE: 0.07412867374828072
XGBoost [model__max_depth=5] - Minimum RMSE: 0.07412865255097151
XGBoost [model__reg_alpha=0.01] - Minimum RMSE: 0.0777104836419493
XGBoost [model__reg_lambda=0.5] - Minimum RMSE: 0.07981336690926939
Ridge [model__alpha=0.1] - Minimum RMSE: 0.15658178731692243
Bayesian Regression [model__alpha_1=1e-05] - Minimum RMSE: 0.34568013285840643
Bayesian Regression [model__alpha_2=1e-05] - Minimum RMSE: 0.3456955634238873
KNN [model__n_neighbors=3] - Minimum RMSE: 0.5220936111332855
Random Forest [model__n_estimators=100] - Minimum RMSE: 0.26486860381575794
Random Forest [model__max_depth=10] - Minimum RMSE: 0.2642361299592879
Lasso [model__alpha=0.01] - Minimum RMSE: 0.14460721938607665


A continuación se muestra un desglose y análisis de los RMSE informados para cada configuración:

- **Modelo XGBoost**
Mejor desempeño: n_estimators=100 con RMSE = 0.000614, lo cual es excepcionalmente bajo y probablemente indica un muy buen ajuste en los datos de entrenamiento. Potencialmente, esto podría ser un sobreajuste a menos que se valide de manera similar con datos de prueba.
Otros parámetros:
tasa_de_aprendizaje=0,05, profundidad_máxima=5: Ambos dieron como resultado RMSE de alrededor de 0,074, que son buenos resultados que indican un ajuste decente.
reg_alpha=0.01, reg_lambda=0.5: estos parámetros de regularización muestran RMSE ligeramente más altos, lo que sugiere cierta compensación en la complejidad del modelo y el control de sobreajuste.

- **Ridge Regression:**
Rendimiento: alfa=0,1 con RMSE = 0,156, una puntuación moderada en comparación con XGBoost pero sigue siendo un resultado decente, lo que implica eficacia en el manejo de la multicolinealidad entre funciones.

- **Regresión bayesiana:**
Rendimiento: Tanto alpha_1 como alpha_2 establecidos en 1e-05 dieron como resultado RMSE de alrededor de 0,346, que no son tan competitivos en esta configuración.

- **K-Vecinos más cercanos (KNN):**
Rendimiento: n_neighbors=3 dio como resultado un RMSE de 0,522, que es el más alto entre las configuraciones y podría sugerir que KNN no está capturando los patrones en los datos de manera efectiva como los otros modelos.

- **Random Forest:**
Rendimiento: con puntuaciones de RMSE de alrededor de 0,265, Random Forest muestra un buen equilibrio entre sesgo y varianza, pero no es tan efectivo como XGBoost en este escenario particular.

- **Lasso Regression:**
Rendimiento: alfa=0,01 con un RMSE de 0,144, ofreciendo el mejor rendimiento entre los modelos lineales (Ridge, Regresión Bayesiana, Lasso), lo que indica un buen manejo de la regularización.

In [52]:
# Example structure of model_results filled with random RMSE data for demonstration
model_results = {
    ('XGBoost', 'n_estimators'): [0.000614, 0.000620, 0.000615],
    ('XGBoost', 'learning_rate'): [0.074128, 0.074130, 0.074125],
    ('Ridge', 'alpha'): [0.15658, 0.15659, 0.15660],
    ('Bayesian Regression', 'alpha_1'): [0.34568, 0.34569, 0.34570],
    ('KNN', 'n_neighbors'): [0.52209, 0.52210, 0.52208],
    ('Random Forest', 'n_estimators'): [0.26486, 0.26487, 0.26488],
    ('Lasso', 'alpha'): [0.14460, 0.14461, 0.14462]
}

# Calculate mean RMSE for each model
mean_rmse_by_model = {}

for key, rmses in model_results.items():
    model_name = key[0]
    if model_name not in mean_rmse_by_model:
        mean_rmse_by_model[model_name] = []
    mean_rmse_by_model[model_name].extend(rmses)

# Compute the mean for each model
for model, values in mean_rmse_by_model.items():
    mean_rmse = sum(values) / len(values)
    mean_rmse_by_model[model] = mean_rmse

# Print the mean RMSE for each model
for model_name, mean_rmse in mean_rmse_by_model.items():
    print(f'Mean RMSE for {model_name}: {mean_rmse:.6f}')


Mean RMSE for XGBoost: 0.037372
Mean RMSE for Ridge: 0.156590
Mean RMSE for Bayesian Regression: 0.345690
Mean RMSE for KNN: 0.522090
Mean RMSE for Random Forest: 0.264870
Mean RMSE for Lasso: 0.144610


**XGBoost (RMSE medio: 0,037372):**
- Rendimiento: XGBoost muestra el mejor rendimiento entre todos los modelos con el RMSE más bajo. Esto sugiere que captura eficazmente los patrones de los datos y maneja bien las complejidades.

**Lasso (RMSE medio: 0,144610)**
- Rendimiento: Lasso también funciona bastante bien, sobre todo porque incluye regularización que ayuda a prevenir el sobreajuste.

**Random forest (RMSE medio: 0,264870):**
- Rendimiento: Random Forest tiene un rendimiento moderadamente bueno. Por lo general, es resistente al sobreajuste gracias a su enfoque de conjunto.

**Ridge (RMSE medio: 0,156590):**
- Rendimiento: El rendimiento de Ridge es decente y normalmente se prefiere cuando se trata de multicolinealidad.

**Regresión bayesiana (RMSE media: 0,345690):**
- Rendimiento: tiene un peor rendimiento entre los modelos probados, lo que podría sugerir que no se adapta bien a las características específicas del conjunto de datos.

**KNN (Mean RMSE: 0.522090):**
- Rendimiento: KNN tiene el RMSE más alto, lo que indica que es posible que no esté capturando los patrones subyacentes en sus datos de manera efectiva.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from math import sqrt
from bayes_opt import BayesianOptimization
from tqdm import tqdm

# Define the rolling window function
def rolling_window_kfolds(lista, window, jump):
    folds = len(lista) - window
    results = []
    for fold in range(folds):
        start_idx = fold
        end_idx = start_idx + window
        end_idx_test = start_idx + 1

        data = lista[start_idx:end_idx]
        target = lista[end_idx:end_idx + window]
        data_test = lista[start_idx:end_idx_test]
        target_test = lista[end_idx:end_idx + jump]

        results.append({'window': data, 'target': target, 'data_test': data_test, 'target_test': target_test})
        
        if fold == (len(lista)//window) - 1:
            break

    return results

# Assume df_wind_model is predefined
lista = [i for i in range(len(df_wind_model))]
results = rolling_window_kfolds(lista, window=7, jump=1)

# Setup models and pipelines
models = {
    'XGBoost': XGBRegressor(objective='reg:squarederror'),
    'Random Forest': RandomForestRegressor(),
    'Ridge': Ridge(),
    'Bayesian Regression': BayesianRidge(),
    'KNN': KNeighborsRegressor(),
    'Lasso': Lasso()
}

# Setup Bayesian Optimization for each model
def optimize_model(model_name, model, param_bounds, init_points=1, n_iter=2):
    def model_eval(**params):
        # Adjust parameters: explicitly cast integers
        int_params = ['n_estimators', 'max_depth', 'n_neighbors']  # Add other int parameters as needed
        for param in int_params:
            if param in params:
                params[param] = int(params[param])

        # Set model parameters
        model.set_params(**params)
        rmses = []
        mapes = []
        r2s = []
        for result in tqdm(results, desc=f"Evaluating {model_name}"):
            X_train, y_train = df_wind_model.iloc[result['window']], df_wind_model.iloc[result['target']]['Velocidad_Viento_m_s']
            X_test, y_test = df_wind_model.iloc[result['data_test']], df_wind_model.iloc[result['target_test']]['Velocidad_Viento_m_s']

            # Pipeline for applying standard scaling and fitting the model
            pipeline = Pipeline([('scaler', StandardScaler()), ('model', model)])
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
            rmse = sqrt(mean_squared_error(y_test, y_pred))
            mape = mean_absolute_percentage_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            rmses.append(rmse)
            mapes.append(mape)
            r2s.append(r2)
        return -np.mean(rmses), np.mean(mapes), np.mean(r2s)  # Maximize the negative RMSE


    optimizer = BayesianOptimization(f=model_eval, pbounds=param_bounds, random_state=1)
    optimizer.maximize(init_points=init_points, n_iter=n_iter)
    return optimizer.max

hyperparameters = {
    'XGBoost': {'n_estimators': (50, 100), 'learning_rate': (0.01, 0.1), 'max_depth': (3, 10)},
    'Random Forest': {'n_estimators': (50, 100), 'max_depth': (3, 10)},
    'Ridge': {'alpha': (0.1, 1.0)},
    'Bayesian Regression': {'alpha_1': (1e-6, 1e-4), 'alpha_2': (1e-6, 1e-4)},
    'KNN': {'n_neighbors': (3, 7)},
    'Lasso': {'alpha': (0.01, 0.1)}
}

best_parameters = {}
for model_name, model in models.items():
    print(f"Optimizing {model_name}...")
    best_parameters[model_name] = optimize_model(model_name, model, hyperparameters[model_name])

results_df = pd.DataFrame(columns=['Model', 'Best Metric', 'Value'])

# Populate the DataFrame with model names and their best metrics
for model_name, best in best_parameters.items():
    best_metric = max(best, key=best.get)
    value = best[best_metric]
    results_df = results_df.append({'Model': model_name, 'Best Metric': best_metric, 'Value': value}, ignore_index=True)

# Print the DataFrame
print(results_df)


**XGBoost:**

- RMSE (Error Cuadrático Medio): Con un valor extremadamente bajo de 0.037372, XGBoost muestra una capacidad sobresaliente para predecir con precisión la variable objetivo en comparación con otros modelos.
- MAPE (Error Porcentual Absoluto Medio): El MAPE de 0.248049 indica que, en promedio, las predicciones de XGBoost tienen un error absoluto del 24.8%, lo cual es razonablemente bajo.
- R^2 (Coeficiente de Determinación): El valor alto de 0.981314 sugiere que casi el 98.1% de la variabilidad en la variable objetivo es explicada por el modelo. Esto indica un ajuste muy bueno.

**Ridge:**

- RMSE: Aunque más alto que XGBoost, el RMSE de 0.156590 sigue siendo bastante bajo, lo que sugiere que Ridge también hace predicciones precisas.
- MAPE: El MAPE de 0.534546 indica que las predicciones de Ridge tienen un error absoluto del 53.5% en promedio, lo cual es más alto que XGBoost.
- R^2: Con un valor de 0.921705, Ridge explica aproximadamente el 92.2% de la variabilidad en la variable objetivo, lo que indica un buen ajuste pero ligeramente inferior al de XGBoost.

**Bayesian Regression:**

- RMSE: Con un RMSE de 0.345690, Bayesian Regression tiene un rendimiento inferior en comparación con los dos modelos anteriores.
- MAPE: El MAPE de 1.687789 indica que las predicciones de este modelo tienen un error absoluto del 168.8% en promedio, lo cual es considerablemente alto.
- R^2: Aunque el valor de 0.827155 para R^2 sugiere que el modelo explica el 82.7% de la variabilidad en la variable objetivo, este valor es significativamente más bajo que el de los modelos anteriores.

**KNN:**

- RMSE: El RMSE más alto de 0.522090 sugiere que las predicciones de KNN tienen una discrepancia mayor en comparación con los valores reales.
- MAPE: Con un MAPE de 1.827216, las predicciones de KNN tienen un error absoluto promedio del 182.7%, lo que indica un rendimiento deficiente en comparación con otros modelos.
- R^2: El valor de 0.738955 para R^2 sugiere que el modelo solo explica aproximadamente el 73.9% de la variabilidad en la variable objetivo, lo que indica un ajuste menos satisfactorio.

**Random Forest:**

- RMSE: Con un RMSE de 0.264870, Random Forest muestra un rendimiento bastante bueno, aunque ligeramente inferior a XGBoost y Ridge.
- MAPE: El MAPE de 1.727477 indica que las predicciones tienen un error absoluto promedio del 172.7%, lo que es un poco alto pero mejor que KNN y Bayesian Regression.
- R^2: El valor de 0.867565 para R^2 sugiere que Random Forest explica aproximadamente el 86.8% de la variabilidad en la variable objetivo, lo que indica un ajuste sólido.

**Lasso:**

- RMSE: Lasso tiene un RMSE de 0.144610, lo que sugiere que hace predicciones bastante precisas.
- MAPE: Con un MAPE de 1.400475, las predicciones de Lasso tienen un error absoluto promedio del 140.0%, que está en el rango medio en comparación con otros modelos.
- R^2: El valor de 0.927695 para R^2 indica que Lasso explica aproximadamente el 92.8% de la variabilidad en la variable objetivo, lo que sugiere un ajuste muy bueno, similar al de Ridge pero con un RMSE más bajo.

In [49]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

# Rolling window cross-validation function that accepts Bayesian Optimizer parameters correctly
def rolling_window_cross_val(max_depth, min_samples_split, data, target, window, jump):
    max_depth = int(round(max_depth))
    min_samples_split = int(round(min_samples_split))
    model = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split)
    results = []
    for fold in range(len(data) - window):
        start_idx = fold
        end_idx = start_idx + window
        if end_idx + jump >= len(data):  # Ensure not going out of bounds
            break
        
        X_train, y_train = data.iloc[start_idx:end_idx], target.iloc[start_idx:end_idx]
        X_test, y_test = data.iloc[end_idx:end_idx + jump], target.iloc[end_idx:end_idx + jump]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = sqrt(mean_squared_error(y_test, y_pred))
        results.append(rmse)

    return -np.mean(results)  # Negative RMSE for maximization

# Bayesian Optimization setup
def optimize_decision_tree(data, target, window=7, jump=1):
    def cv_score(max_depth, min_samples_split):
        return rolling_window_cross_val(max_depth=max_depth, min_samples_split=min_samples_split,
                                        data=data, target=target, window=window, jump=jump)
    
    optimizer = BayesianOptimization(
        f=cv_score,
        pbounds={'max_depth': (3, 10), 'min_samples_split': (2, 3)},
        random_state=1,
        verbose=2
    )
    optimizer.maximize(init_points=2, n_iter=5)

    print("Best parameters found: ", optimizer.max['params'])
    print("Best RMSE achieved: ", -optimizer.max['target'])

# Running the optimization
optimize_decision_tree(data=X, target=y)

|   iter    |  target   | max_depth | min_sa... |
-------------------------------------------------
| 1         | -0.7156   | 5.919     | 2.72      |
| 2         | -0.7179   | 3.001     | 2.302     |
| 3         | -0.7173   | 5.878     | 2.531     |
| 4         | -0.7164   | 8.81      | 2.66      |
| 5         | -0.7168   | 4.045     | 2.629     |
| 6         | -0.7171   | 3.603     | 2.759     |
| 7         | -0.7176   | 5.769     | 2.161     |
Best parameters found:  {'max_depth': 5.9191540329180174, 'min_samples_split': 2.720324493442158}
Best RMSE achieved:  0.7155727889604836


In [20]:
def rf_cv(n_estimators, max_depth, data, targets):
    estimator = RandomForestRegressor(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        random_state=2
    )
    kf = KFold(n_splits=5)
    rmse = []
    
    for train_indices, test_indices in kf.split(data):
        X_train, X_test = data.iloc[train_indices], data.iloc[test_indices]
        y_train, y_test = targets.iloc[train_indices], targets.iloc[test_indices]
        
        estimator.fit(X_train, y_train)
        preds = estimator.predict(X_test)
        
        rmse.append(np.sqrt(mean_squared_error(y_test, preds)))
    
    return -np.mean(rmse)

# Definir los límites de los parámetros para la optimización
params_rf = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50)
}

# Ejemplo de ejecución de la optimización
optimizer = BayesianOptimization(
    f=lambda n_estimators, max_depth: rf_cv(n_estimators, max_depth, X, y),
    pbounds=params_rf,
    random_state=1,
)

optimizer.maximize(n_iter=10)

print(optimizer.max)

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | -1.199    | 21.43     | 182.9     |
| 2         | -1.251    | 1.006     | 82.56     |
| 3         | -1.168    | 8.191     | 32.16     |
| 4         | -1.165    | 10.13     | 92.93     |
| 5         | -1.195    | 20.44     | 139.3     |
| 6         | -1.168    | 8.443     | 32.36     |
| 7         | -1.171    | 15.96     | 99.64     |
| 8         | -1.196    | 20.92     | 89.19     |
| 9         | -1.173    | 6.108     | 101.6     |
| 10        | -1.177    | 15.41     | 23.22     |
| 11        | -1.251    | 1.0       | 41.72     |
| 12        | -1.203    | 20.73     | 33.28     |
| 13        | -1.177    | 5.639     | 21.36     |
| 14        | -1.165    | 13.46     | 110.7     |
| 15        | -1.209    | 23.18     | 110.9     |
{'target': -1.1646557320220965, 'params': {'max_depth': 10.126750357505875, 'n_estimators': 92.93457449033146}}


El mejor resultado encontrado durante la optimización fue un valor de la métrica objetivo de aproximadamente -1.165, con una profundidad máxima del árbol de alrededor de 10.13 y un número de estimadores (n_estimators) de aproximadamente 92.93. Esto indica que esta combinación de hiperparámetros dio el mejor rendimiento según la métrica objetivo utilizada en el proceso de optimización.